In [1]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr


In [5]:
from dask_jobqueue import SLURMCluster

cluster = SLURMCluster(
    job_name="Climt1",          # --job-name
    cores=2,                     # Number of cores per task (adjust if needed)
    processes=5,                 # One process per task
    memory="100GB",               # --mem
    walltime="00:15:00",         # --time
    queue="med",               # --partition
    log_directory=".",           # Logs will be saved to the current directory
)


/home1/nalex2023/.conda/envs/nma/lib/python3.13/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 35779 instead
  warnings.warn(


In [7]:
from dask.distributed import Client

# Scale up the number of workers to 10
cluster.scale(10)

client = Client(cluster)    

In [8]:
client

<Client: 'tcp://10.42.239.61:39311' processes=10 threads=20, memory=186.30 GiB>

In [10]:
import glob
import pandas as pd 

sst_dsets = glob.glob('/home1/nalex2023/Datasets/sst_datasets/*.nc')


In [4]:
prefix = '/home1/nalex2023/Datasets/LSB_OUTS/'

era5_climt_file = prefix + 'era5_manus_diurnal.nc'

era5_climt = xr.open_dataset(era5_climt_file)



<xarray.Dataset> Size: 1MB
Dimensions:         (hour: 24, pressure_level: 3, latitude: 29, longitude: 41)
Coordinates:
    number          int64 8B ...
  * pressure_level  (pressure_level) float64 24B 1e+03 900.0 800.0
  * latitude        (latitude) float64 232B 2.0 1.75 1.5 ... -4.5 -4.75 -5.0
  * longitude       (longitude) float64 328B 140.0 140.2 140.5 ... 149.8 150.0
  * hour            (hour) int64 192B 0 1 2 3 4 5 6 7 ... 17 18 19 20 21 22 23
Data variables:
    u               (hour, pressure_level, latitude, longitude) float32 342kB ...
    v               (hour, pressure_level, latitude, longitude) float32 342kB ...
    speed           (hour, pressure_level, latitude, longitude) float32 342kB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-12-23T21:11 GRIB to CDM+CF via cfgrib-0.9.1...

In [14]:
import os 
sst_files = sst_dsets

sst_files_df = pd.DataFrame(sst_files,columns=['filepath'])

sst_files_df['filename'] = sst_files_df['filepath'].str.split(os.sep).str[-1]

sst_files_df['datetime'] = sst_files_df['filename'].str.split('.').str[-2]

test = xr.open_dataset(sst_files_df.iloc[22]['filepath'])
sst_files_df['datetime'] = sst_files_df['datetime'].str.replace(" (1)",'')



In [19]:
lats = era5_climt.latitude[0].values, era5_climt.latitude[-1].values
lons = era5_climt.longitude[0].values, era5_climt.longitude[-1].values


def prepro(ds):
    ds_sel = ds.sel(lat=slice(lats[0],lats[1]),lon=slice(lons[0],lons[1]))
    return ds_sel

sst_all = xr.open_mfdataset(sst_files_df['filepath'].values,preprocess=prepro,combine='by_coords',
                            parallel=True)

KeyboardInterrupt: 